In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
links=pd.read_excel("links.xlsx")


In [3]:
links

,Dados,Links,Anual,Mensal,Diário
0,IPCA,http://www.ipeadata.gov.br/ExibeSerie.aspx?ser...,NaN,Sim,NaN
1,PIB,http://www.ipeadata.gov.br/ExibeSerie.aspx?ser...,NaN,Sim,NaN
2,INPC,http://www.ipeadata.gov.br/ExibeSerie.aspx?ser...,NaN,Sim,NaN
3,Dólar,http://www.ipeadata.gov.br/ExibeSerie.aspx?ser...,NaN,Sim,NaN
4,Salário mínimo vigente,http://www.ipeadata.gov.br/ExibeSerie.aspx?ser...,NaN,Sim,NaN
5,Desemprego (até 2016.02),http://www.ipeadata.gov.br/ExibeSerie.aspx?ser...,NaN,Sim,NaN
6,CDI,http://www.ipeadata.gov.br/ExibeSerie.aspx?ser...,NaN,Sim,NaN
7,Taxa de juros nominal Overnight Selic,http://www.ipeadata.gov.br/ExibeSerie.aspx?ser...,NaN,Sim,NaN
8,IBOVESPA fechamento mensal,http://www.ipeadata.gov.br/ExibeSerie.aspx?ser...,NaN,Sim,NaN
9,IBOVESPA fechamento diário,http://www.ipeadata.gov.br//ExibeSerie.aspx?se...,NaN,NaN,Sim


In [4]:
def get_data_ipeadata(link):
    """ 
    str => list
    Recebe uma url do site ipeadata, e retorna a tabela com os dados
    Utiliza a biblioteca requests e beautiful soup
    A tabela é encontrada por meio da tag <td> e da classe "dxgv"
    """

    #pega html do site
    html=requests.get(link).content

    #tratamento com o BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    #filtra o que queremos, que é o que está dentro da tag <td> com a classe "dxgv", a parte string=True garante que não vamos pegar strings vazias
    td_tags = soup.find_all('td',class_="dxgv",string=True)

    #extrai o texto de dentro da tag, que é o dado que queremos mesmo, e coloca numa lista
    tds = [td.text for td in td_tags]
   
    
    return tds

def transform_data(dado,lista,mensal,diario):
    """ 
    str,list,str,str => dataframe
    Recebe o nome do dado, a lista com as tags, e se o dado é mensal ou não
    Transforma a lista num dataframe de 2 colunas, uma com a data e outra com o dado daquela data
    Se for mensal, cria duas outras colunas com o ano e o mês (em formato de número)
    Se for diário, cria três outras colunas com o ano, o mês(em formato de número) e o dia
    Também cria uma coluna com a variação percentual dos dados
    """
    #transforma a lista obtida em um dataframe com duas colunas, e define o nome das colunas
    df=pd.DataFrame([lista[n:n+2] for n in range(0, len(lista), 2)])
    df.columns=["data",dado]
    
    #corrige pontuação das casas de milhar e da vírgula, além de transformar o tipo do dado para númerico (isso é util para quando formos exportar para csv)
    df[dado]=pd.to_numeric(df[dado].apply(lambda x: str(x).strip().replace(".","").replace(",",".")))
    #separa mês do ano
    if mensal == "Sim": df[["ano", "mês"]] = df["data"].str.split(".", expand = True)
    elif diario == "Sim": df[["dia", "mês","ano"]] = df["data"].str.split("/", expand = True)
    
    #calcula variação percentual
    df["variação"]= df[dado].pct_change()
    return df

In [7]:
dfs={}
for index,row in links.iterrows():
    dado=links.loc[index,"Dados"]
    link=links.loc[index,"Links"]
    mensal,diario=links.loc[index,"Mensal"],links.loc[index,"Diário"]
    lista=get_data_ipeadata(link)
    df=transform_data(dado,lista,mensal,diario)
    dfs[dado]=df
    df.to_csv(f"./databases/{dado}.csv",sep=";",decimal=",")

In [8]:
dfs['IPCA']

,data,IPCA,ano,mês,variação
0,1979.12,7.618300e-09,1979,12,NaN
1,1980.01,8.122300e-09,1980,01,0.066156
2,1980.02,8.497300e-09,1980,02,0.046169
3,1980.03,9.010400e-09,1980,03,0.060384
4,1980.04,9.486700e-09,1980,04,0.052861
...,...,...,...,...,...
513,2022.09,6.370340e+03,2022,09,-0.002900
514,2022.10,6.407930e+03,2022,10,0.005901
515,2022.11,6.434200e+03,2022,11,0.004100
516,2022.12,6.474090e+03,2022,12,0.006200


In [9]:
dfs

{'IPCA':         data          IPCA   ano mês  variação
 0    1979.12  7.618300e-09  1979  12       NaN
 1    1980.01  8.122300e-09  1980  01  0.066156
 2    1980.02  8.497300e-09  1980  02  0.046169
 3    1980.03  9.010400e-09  1980  03  0.060384
 4    1980.04  9.486700e-09  1980  04  0.052861
 ..       ...           ...   ...  ..       ...
 513  2022.09  6.370340e+03  2022  09 -0.002900
 514  2022.10  6.407930e+03  2022  10  0.005901
 515  2022.11  6.434200e+03  2022  11  0.004100
 516  2022.12  6.474090e+03  2022  12  0.006200
 517  2023.01  6.508400e+03  2023  01  0.005300
 
 [518 rows x 5 columns],
 'PIB':         data       PIB   ano mês  variação
 0    1990.01       0.2  1990  01       NaN
 1    1990.02       0.4  1990  02  1.000000
 2    1990.03       0.8  1990  03  1.000000
 3    1990.04       0.7  1990  04 -0.125000
 4    1990.05       0.8  1990  05  0.142857
 ..       ...       ...   ...  ..       ...
 392  2022.09  829534.3  2022  09 -0.030885
 393  2022.10  832985.7  2022 